In [1]:
!uv pip install unsloth transformers torch bitsandbytes peft accelerate datasets

Using Python 3.11.13 environment at: /home/ardjano/.pyenv/versions/3.11.13/envs/unsloth-env
Audited 7 packages in 8ms


In [6]:
!uv pip install ipywidgets scikit-learn

Using Python 3.11.13 environment at: /home/ardjano/.pyenv/versions/3.11.13/envs/unsloth-env
Resolved 26 packages in 24ms                                         
Installed 4 packages in 23ms                                
 + joblib==1.5.1
 + scikit-learn==1.7.1
 + scipy==1.16.1
 + threadpoolctl==3.6.0


In [ ]:
!uv pip install matplotlib seaborn

In [2]:
from unsloth import FastModel
import torch
import pandas as pd
import numpy as np
from datasets import Dataset
from trl import SFTTrainer, SFTConfig
from unsloth.chat_templates import train_on_responses_only
from unsloth.chat_templates import get_chat_template

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/home/ardjano/.pyenv/versions/unsloth-env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!


In [4]:
# more: https://huggingface.co/unsloth
fourbit_models = [
    "unsloth/gemma-3-4b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-12b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-27b-it-unsloth-bnb-4bit",
]
# "unsloth/gemma-3-4b-it"
model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3-4b-it-unsloth-bnb-4bit",
    max_seq_length = 2048,
    load_in_4bit = True,
    load_in_8bit = False,
    full_finetuning = False,
)

==((====))==  Unsloth 2025.7.11: Fast Gemma3 patching. Transformers: 4.54.1.
   \\   /|    NVIDIA GeForce RTX 3090. Num GPUs = 1. Max memory: 23.683 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 8.6. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.31.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [7]:
model = FastModel.get_peft_model(
    model,
    finetune_vision_layers = False,
    finetune_language_layers = True,
    finetune_attention_modules = True,
    finetune_mlp_modules = True,

    # larger = higher accuracy, but might overfit -> maybe create tabel testing various r numbers
    # find recommendations of it
    r = 8,

    # recommended: alpha == r, at least
    lora_alpha = 8,
    lora_dropout = 0,
    bias = "none",
    random_state = 42,
)

Unsloth: Making `model.base_model.model.model.language_model` require gradients


In [8]:
from sklearn.model_selection import train_test_split

df = pd.read_csv("../data/humanandllm.csv")
df = df[['Context', 'Response']]

trainset, testset = train_test_split(df, test_size=0.057, random_state=42)

In [10]:
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)

def format_chat_template(row):
    """
    Formats the context and response into a single string
    using the tokenizer's chat template.
    """
    # Create the conversation structure
    row_json = [
        {"role": "user", "content": row["Context"]},
        {"role": "assistant", "content": row["Response"]}
    ]
    # Apply the template and remove the beginning-of-string token
    row["text"] = tokenizer.apply_chat_template(row_json, tokenize=False, add_generation_prompt=False).replace(tokenizer.bos_token, "")
    return row



In [14]:
# Apply the formatting function to each split DataFrame
formatted_trainset = trainset.apply(format_chat_template, axis=1)
formatted_testset = testset.apply(format_chat_template, axis=1)

# Convert each pandas DataFrame to a Hugging Face Dataset
hf_train = Dataset.from_pandas(formatted_trainset)
hf_test = Dataset.from_pandas(formatted_testset)

hf_train = hf_train.remove_columns(["Context", "Response"])
hf_test = hf_test.remove_columns(["Context", "Response"])

{'text': "<start_of_turn>user\nI'm being verbally abused on a daily basis by my parents in front of my child. I feel like I’m trapped and I have no escape. I feel like I have to listen to the horrible things they say and just take all of it.\n   I need some way of learning how to cope with listening to it especially since I have nowhere else to go. I feel very emotionally drained How can I deal with this?<end_of_turn>\n<start_of_turn>model\nWhen their verbal abuse starts, tell them you are exiting the conversation because the way they're talking to you is unacceptable.Explain you are willing to hear their point of view only not when it is expressed as abuse.They may not agree with your opinion and also may not want to change.Even if they do not change, you are entitled to be treated as a human being who is worthy of respect.Explain your reason to exit the conversation and ask them to write down their requests for you to consider.<end_of_turn>\n",
 '__index_level_0__': 612}

In [21]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = hf_train,
    eval_dataset  = hf_test,
    args = SFTConfig(
        dataset_text_field          = "text",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps                = 5,
        num_train_epochs          = 1, # maybe look at it later
        # max_steps                   = 30,
        learning_rate               = 2e-4,
        logging_steps               = 5,

        # evaluation
        eval_strategy               = "steps",
        eval_steps                  = 10,
        save_strategy               = "steps",
        save_steps                  = 10,
        save_total_limit            = 2,

        load_best_model_at_end      = True,

        optim                       = "adamw_8bit",
        weight_decay                = 0.01,
        lr_scheduler_type           = "linear",
        seed                        = 42,
        report_to                   = "none",
        dataset_num_proc            = 2,
    )
)

Unsloth: Tokenizing ["text"] (num_proc=2): 100%|██████████| 200/200 [00:01<00:00, 131.04 examples/s]


In [17]:
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<start_of_turn>user\n",
    response_part = "<start_of_turn>model\n",
)

Map (num_proc=20): 100%|██████████| 200/200 [00:00<00:00, 843.42 examples/s]


In [29]:
print("--- Getting Baseline Performance ---")
baseline_metrics = trainer.evaluate()
print(baseline_metrics)


--- Getting Baseline Performance ---
{'eval_loss': 2.0635826587677, 'eval_model_preparation_time': 0.0077}


In [ ]:
torch._dynamo.config.cache_size_limit = 64

In [23]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 3,307 | Num Epochs = 1 | Total steps = 414
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 14,901,248 of 4,314,980,720 (0.35% trained)


Step,Training Loss,Validation Loss,Model Preparation Time
10,2.512200,2.443262,0.007700
20,2.176200,2.374247,0.007700
30,2.285100,2.333491,0.007700
40,2.228700,2.298296,0.007700
50,2.091800,2.284208,0.007700
60,2.226400,2.266951,0.007700
70,2.131600,2.253704,0.007700
80,2.231200,2.242753,0.007700
90,2.169100,2.230362,0.007700
100,2.131000,2.221341,0.007700


FailOnRecompileLimitHit: recompile_limit reached with one_graph=True. Excessive recompilations can degrade performance due to the compilation overhead of each recompilation. To monitor recompilations, enable TORCH_LOGS=recompiles. If recompilations are expected, consider increasing torch._dynamo.config.cache_size_limit to an appropriate value.

In [31]:
baseline_eval_loss_at_step_0 = 2.8692877292633057

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns

# Apply the professional seaborn theme
sns.set_theme(style="whitegrid", context="paper")

log_history = trainer.state.log_history
train_logs = [log for log in log_history if 'loss' in log]
eval_logs = [log for log in log_history if 'eval_loss' in log]

train_steps = [log['step'] for log in train_logs]
train_losses = [log['loss'] for log in train_logs]
eval_steps = [log['step'] for log in eval_logs]
eval_losses = [log['eval_loss'] for log in eval_logs]

eval_steps.insert(0, 0)
eval_losses.insert(0, baseline_eval_loss_at_step_0)

# Create the plot
plt.figure(figsize=(10, 6))

# sns.lineplot for styled lines
sns.lineplot(x=train_steps, y=train_losses, label='Training Loss')
sns.lineplot(x=eval_steps, y=eval_losses, label='Validation Loss', marker='o', linestyle='--')

plt.title('Training vs. Validation Loss')
plt.xlabel('Training Steps')
plt.ylabel('Loss')
plt.legend()
plt.savefig('gemma_loss.pdf')
plt.show()

NameError: name 'trainer' is not defined

In [24]:
trainer.save_model("../trained_models/gemma3")

In [25]:
model.save_pretrained_merged(
    "../trained_models/gemma3-finetuned-16bitv1",  # A new folder name for the final model
    tokenizer,
    save_method = "merged_16bit",
)

Found HuggingFace hub cache directory: /home/ardjano/.cache/huggingface/hub
Checking cache directory for required files...
Cache check failed: model-00001-of-00002.safetensors not found in local cache.
Not all required files found in cache. Will proceed with downloading.


Unsloth: Merging weights into 16bit: 100%|██████████| 2/2 [11:15<00:00, 337.73s/it]
